In [1]:
import neo4j
from neo4j import GraphDatabase

In [2]:
class HelloWorld(object):
    
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth = (user,password))
        
    def close(self):
        self._driver.close()

    def run_query(self, query):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._run_query, query)
            print(greeting)
            return greeting

    @staticmethod
    def _run_query(tx, query):
        result = tx.run(query)
        return result

In [3]:
neo = HelloWorld('bolt://localhost:7687', 'neo4j', 'test123')

In [4]:
result = neo.run_query('MATCH (n) MATCH ()-[r]->() RETURN n')

In [5]:
result.peek()

In [6]:
import psycopg2
import time

In [7]:
host = 'db.talentify.in'
password = 'cx6ac54nmgGtLD1y'
start = time.time()
sql = 'select analytics from task where id = 17949830'
try:
    conn = psycopg2.connect("dbname='sales' user='postgres' host='"+host+"' password='"+password+"'")
    print("Database Connection established successsfully after: "+str(time.time()-start))
except:
    print('Establishing connection with db failed')
    raise
cur = conn.cursor()
try:
    cur.execute(sql)
except:
    print("Query fetch failed")
    raise
rows = cur.fetchall()
print("Fetched "+str(len(rows))+" tasks after: "+str(time.time()-start))

Database Connection established successsfully after: 0.20848822593688965
Fetched 1 tasks after: 0.41449975967407227


In [9]:
import json
ana = json.loads(rows[0][0])

In [14]:
for index, conv in enumerate(ana['conversation']):
    print(str(index)+'-->'+conv['speaker']+'-------->>>>>>'+conv['text']) 

0-->Agent-------->>>>>>Hi, I am I speaking to mr. Praveen Matthew.
1-->Customer-------->>>>>>Yeah, this is Praveen . Whos calling?
2-->Agent-------->>>>>>Hello, mr. Praveen. Yeah, good morning. My name is Paul and I''m a learning consultant with eazylearn.
You had you had inquired last week on a website about a digital marketing associate course, and I''m calling.
3-->Agent-------->>>>>>You had you had inquired last week on a website about a digital marketing associate course, and I''m calling.
4-->Customer-------->>>>>>Yeah, yeah.
5-->Agent-------->>>>>>Yeah, I''m calling in connection with that. Is it a good time to talk to you right now?
6-->Customer-------->>>>>>Yeah, yeah. Yeah, please continue. Like I wanted I had inquired about that course.
I want to know about that. Yeah.
7-->Customer-------->>>>>>I want to know about that. Yeah.
8-->Agent-------->>>>>>Sure, before I start with the course details. I just wanted a little bit about our background about yourself. Where are you bas

In [11]:
def clean_analysis(ana):
    """We can have repeating blocks by agents/cusdtomer, should merge them into one"""
    convs = []
    curr_speaker = None
    for conv in ana['conversation']:
        if curr_speaker == None:
            curr_speaker = conv['speaker']
            convs.append(conv)
        else:
            if conv['speaker'] != curr_speaker:
                #print(convs)
                convs.append(conv)
            else:
                #print(convs)
                prev_conv = convs[-1]
                prev_conv['text'] = prev_conv['text'] + '\n' + conv['text']
                convs[-1] = prev_conv
            curr_speaker = conv['speaker']
    return convs

In [12]:
convss = clean_analysis(ana)

In [13]:
for index, conv in enumerate(convss):
    print(str(index)+'-->'+conv['speaker']+'-------->>>>>>'+conv['text']) 

0-->Agent-------->>>>>>Hi, I am I speaking to mr. Praveen Matthew.
1-->Customer-------->>>>>>Yeah, this is Praveen . Whos calling?
2-->Agent-------->>>>>>Hello, mr. Praveen. Yeah, good morning. My name is Paul and I''m a learning consultant with eazylearn.
You had you had inquired last week on a website about a digital marketing associate course, and I''m calling.
3-->Customer-------->>>>>>Yeah, yeah.
4-->Agent-------->>>>>>Yeah, I''m calling in connection with that. Is it a good time to talk to you right now?
5-->Customer-------->>>>>>Yeah, yeah. Yeah, please continue. Like I wanted I had inquired about that course.
I want to know about that. Yeah.
6-->Agent-------->>>>>>Sure, before I start with the course details. I just wanted a little bit about our background about yourself. Where are you based at the moment&amp;amp;amp;amp;amp;amp;nbsp;
7-->Customer-------->>>>>>I'm I''m I''m in Bangalore right now.
8-->Agent-------->>>>>>You're in Bangalore. Okay, that''s great. 
beautiful weath

In [15]:
ana['conversation_clean'] = convss

In [16]:
def find_previous_agent(index, convs):
    prev = None
    for i, conv in enumerate(convs):
        if i == index:
            break
        prev = conv
    return prev

def find_next_agent(index, convs):
    found = False
    next_block = None
    for i, conv in enumerate(convs):
        next_block = conv
        if found: 
            break
        if i == index:
            found = True
    return next_block

In [63]:
import re
def clean_text_cypher(text):
    p = re.compile("[^a-zA-Z\\s]")
    text= p.sub('', text)
    text = text.replace('\n', ' ').replace(" ","_")
    text = text[:100]
    return text

In [64]:
print(clean_text_cypher('Some randinm}&554 text but very long and heavy \n so ther eneeds sto be '))

Some_randinm_text_but_very_long_and_heavy___so_ther_eneeds_sto_be_


In [73]:
cypher = ''
for indd, conv in enumerate(ana['conversation_clean']):
    if conv['speaker'] == 'Agent':
        cypher += 'create (n'+str(indd)+':agent{message:"'+clean_text_cypher(conv["text"])+'"})'
for indq, conv in enumerate(ana['conversation_clean']):
    if conv['speaker'] == 'Customer':
        cypher += 'create(n'+str(indq-1)+')-[r'+str(indq)+':'+clean_text_cypher(conv["text"])+']->(n'+str(indq+1)+')'+' '
        #cypher += 'create (n'+str(prev['index'])+':agent{message:"'+conv["text"].replace(' ','_')+'"})'

In [74]:
neo.run_query(cypher)

In [69]:
cypher

'create (n0:agent{message:"Hi_I_am_I_speaking_to_mr_Praveen_Matthew"})create (n2:agent{message:"Hello_mr_Praveen_Yeah_good_morning_My_name_is_Paul_and_Im_a_learning_consultant_with_eazylearn_You_h"})create (n4:agent{message:"Yeah_Im_calling_in_connection_with_that_Is_it_a_good_time_to_talk_to_you_right_now"})create (n6:agent{message:"Sure_before_I_start_with_the_course_details_I_just_wanted_a_little_bit_about_our_background_about_yo"})create (n8:agent{message:"Youre_in_Bangalore_Okay_thats_great__beautiful_weather_right_now_in_Bangalore_its_only__degree_i_bel"})create (n10:agent{message:"and_you_would_have_been_really_excited_that_our_RCB_won_against_CSK_Yesterday_by_just_wondering_"})create (n12:agent{message:"Great_Are_you_a_student_or_are_you_working"})create (n14:agent{message:"Youre_working_Is_it_great_Which_company_do_you_work_with"})create (n16:agent{message:"Of_course_Yes_tonic_cosmetics_And_how_long_have_you_been_with_them_"})create (n18:agent{message:"Great_three_years_with_o

In [71]:
for indd, conv in enumerate(ana['conversation_clean']):
    print(str(indd)+'-->'+conv['speaker']+'-------->>>>>>'+conv['text']) 

0-->Agent-------->>>>>>Hi, I am I speaking to mr. Praveen Matthew.
1-->Customer-------->>>>>>Yeah, this is Praveen . Whos calling?
2-->Agent-------->>>>>>Hello, mr. Praveen. Yeah, good morning. My name is Paul and I''m a learning consultant with eazylearn.
You had you had inquired last week on a website about a digital marketing associate course, and I''m calling.
3-->Customer-------->>>>>>Yeah, yeah.
4-->Agent-------->>>>>>Yeah, I''m calling in connection with that. Is it a good time to talk to you right now?
5-->Customer-------->>>>>>Yeah, yeah. Yeah, please continue. Like I wanted I had inquired about that course.
I want to know about that. Yeah.
6-->Agent-------->>>>>>Sure, before I start with the course details. I just wanted a little bit about our background about yourself. Where are you based at the moment&amp;amp;amp;amp;amp;amp;nbsp;
7-->Customer-------->>>>>>I'm I''m I''m in Bangalore right now.
8-->Agent-------->>>>>>You're in Bangalore. Okay, that''s great. 
beautiful weath

In [52]:
neo.run_query(cypher)

CypherSyntaxError: Invalid input 'I': expected whitespace, comment, '|', a length specification, a property map or ']' (line 4, column 1 (offset: 182))
"I_want_to_know_about_t]->(n7)"
 ^

In [53]:
cypher

'create(n0)-[r1:Yeah_this_is_Praveen__Whos_calling]->(n2)\ncreate(n3)-[r4:Yeah_yeah]->(n5)\ncreate(n5)-[r6:Yeah_yeah_Yeah_please_continue_Like_I_wanted_I_had_inquired_about_that_course\nI_want_to_know_about_t]->(n7)\ncreate(n6)-[r7:I_want_to_know_about_that_Yeah]->(n8)\ncreate(n8)-[r9:Im_Im_Im_in_Bangalore_right_now]->(n10)\ncreate(n11)-[r12:Haha_yeah_but_you_know_still_because_of_all_this_global_warming_Its_getting_hotter_like_day_by_day]->(n13)\ncreate(n13)-[r14:Yeah_II_finally_they_won_again]->(n15)\ncreate(n15)-[r16:Im_working_right_now]->(n17)\ncreate(n17)-[r18:Its_a_company_called_tonic_I_think_you_must_have_you_might_have_heard_about_it\nThey_are_into_the_Cos]->(n19)\ncreate(n18)-[r19:They_are_into_the_Cosmetic_space_and_all_that_you_know_like_they_sell_cosmetic_products_and_all_that]->(n20)\ncreate(n20)-[r21:I_dont_no_its_be_like__years_now_Ive_finished_my_masters_and_I_joined_in_there\nBoth_Ive_been_working]->(n22)\ncreate(n21)-[r22:Both_Ive_been_working_there_for_almost_three